In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import json
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
# URL to scrape
url = 'https://en.m.wikipedia.org/wiki/List_of_Falcon_9_and_Falcon_Heavy_launches_(2010%E2%80%932019)'

response = requests.get(url)
soup = BeautifulSoup(response.content, 'html.parser')

tables = soup.find_all('table')
#print(tables.prettify())

In [11]:
"""
def extract_column_from_header(row):

    This function returns a cleaned column name from an HTML table cell (th).
    It removes unwanted tags like <br>, <sup> but keeps the text from <a> tags and ensures proper spacing.

    # Remove <sup> entirely, as they are references
    for tag in row.find_all(['sup']):
        tag.decompose()  # Remove the tag and its contents

    # Collect all text from the row, ensuring proper spacing but no unnecessary commas
    column_name = ' '.join(row.stripped_strings)

    # Remove any purely numeric or empty headers
    if not column_name.strip().isdigit():
        return column_name.strip()

    return None



url = 'https://en.m.wikipedia.org/wiki/List_of_Falcon_9_and_Falcon_Heavy_launches_(2010%E2%80%932019)'

# Make a request to the website
response = requests.get(url)

# Parse the HTML content
soup = BeautifulSoup(response.content, 'html.parser')

# Find the first table in the page (you can change this based on which table you need)
table = soup.find('table')

# Get all the rows in the table body (tbody)
rows = table.find('tbody').find_all('tr')

# Get the header (first row) of the table
header_row = rows[0]
headers = []

# Extract headers
for th in header_row.find_all('th'):
    column_name = extract_column_from_header(th)
    if column_name:
        headers.append(column_name)

# Print extracted headers
print(headers)
"""

['Flight No.', 'Date and time ( UTC )', 'Version, booster', 'Launch site', 'Payload', 'Payload mass', 'Orbit', 'Customer', 'Launch outcome', 'Booster landing']


In [3]:
"""
def extract_column_from_header(row):

    This function returns a cleaned column name from an HTML table cell (th).
    It removes unwanted tags like <br>, <sup> but keeps the text from <a> tags and ensures proper spacing.

    for tag in row.find_all(['sup']):
        tag.decompose()  # Remove the tag and its contents

    column_name = ' '.join(row.stripped_strings)

    if not column_name.strip().isdigit():
        return column_name.strip()

    return None

def extract_column_from_data(row):

    This function returns cleaned data from an HTML table cell (td).
    It removes unwanted tags but keeps the text from <a> tags and ensures proper spacing.

    # Remove unwanted tags
    for tag in row.find_all(['sup', 'br']):
        tag.decompose()  # Remove the tag and its contents

    # Collect the text from <a> tags and other text
    text_parts = []

    for a in row.find_all('a'):
        text_parts.append(a.get_text(strip=True))

    for text in row.stripped_strings:
        text_parts.append(text)

    unique_text_parts = list(dict.fromkeys(text_parts))
    data_value = ' '.join(part for part in unique_text_parts if part)

    return data_value.strip()

# Define the URL to scrape
url = 'https://en.m.wikipedia.org/wiki/List_of_Falcon_9_and_Falcon_Heavy_launches_(2010%E2%80%932019)'

# Make a request to the website
response = requests.get(url)

# Parse the HTML content
soup = BeautifulSoup(response.content, 'html.parser')

# Find the first table in the page
table = soup.find('table')

# Get all the rows in the table body (tbody)
rows = table.find('tbody').find_all('tr')

# Extract headers
header_row = rows[0]
headers = []

for th in header_row.find_all('th'):
    column_name = extract_column_from_header(th)
    if column_name:
        headers.append(column_name)

# Print extracted headers
print("Headers:", headers)

# Extract data, ensuring valid data rows (those with sufficient <td> elements)
all_data_rows = []
for data_row in rows[1:]:
    data_values = []

    # Check if the row contains a flight number (i.e., it has a <th> element)
    flight_no_tag = data_row.find('th')
    td_tags = data_row.find_all('td')

    # Process only rows that have a <th> (flight number) and sufficient <td> elements (usually around 9-10 data points)
    if flight_no_tag and len(td_tags) >= 9:
        # Extract Flight No. from the first <th> in the row
        flight_no = extract_column_from_data(flight_no_tag)
        data_values.append(flight_no)

        # Extract data from all <td> elements in the row
        for td in td_tags:
            data_value = extract_column_from_data(td)
            data_values.append(data_value)

        # Add the extracted data row to the list
        all_data_rows.append(data_values)

# Print extracted data for each row
for idx, data_row in enumerate(all_data_rows, 1):
    print(f"Row {idx} Data:", data_row)
"""

Headers: ['Flight No.', 'Date and time ( UTC )', 'Version, booster', 'Launch site', 'Payload', 'Payload mass', 'Orbit', 'Customer', 'Launch outcome', 'Booster landing']
Row 1 Data: ['1', '4 June 2010 18:45', 'F9v1.0 B0003 F9 v1.0', 'Cape Canaveral SLC‑40 ,', 'Dragon Spacecraft Qualification Unit', 'Unknown', 'LEO', 'SpaceX', 'Success', 'Failure (parachute)']
Row 2 Data: ['2', '8 December 2010 15:43', 'F9v1.0 B0004 F9 v1.0', 'Cape Canaveral SLC‑40 ,', 'SpaceX COTS Demo Flight 1 (Dragon C101)', 'Unknown', 'LEO ISS ( )', 'NASA COTS ( )', 'Success', 'Failure (parachute)']
Row 3 Data: ['3', '22 May 2012 07:44', 'F9 v1.0 B0005', 'Cape Canaveral SLC‑40 ,', 'SpaceX COTS Demo Flight 2 (Dragon C102)', '525\xa0kg (1,157\xa0lb) (excl. Dragon mass)', 'LEO ISS ( )', 'NASA COTS ( )', 'Success', 'No attempt']
Row 4 Data: ['4', '8 October 2012 00:35', 'F9 v1.0 B0006', 'Cape Canaveral SLC‑40 ,', 'SpaceX CRS-1 (Dragon C103)', '4,700\xa0kg (10,400\xa0lb) (excl. Dragon mass)', 'LEO ISS ( )', 'NASA CRS ( )'

In [5]:
"""
import requests
from bs4 import BeautifulSoup

def extract_column_from_header(row):

    This function returns a cleaned column name from an HTML table cell (th).
    It removes unwanted tags like <br>, <sup> but keeps the text from <a> tags and ensures proper spacing.

    for tag in row.find_all(['sup']):
        tag.decompose()  # Remove the tag and its contents

    column_name = ' '.join(row.stripped_strings)

    if not column_name.strip().isdigit():
        return column_name.strip()

    return None

def extract_column_from_data(row):

    # Remove unwanted tags
    for tag in row.find_all(['sup', 'br']):
        tag.decompose()  # Remove the tag and its contents

    # Collect the text from <a> tags and other text
    text_parts = []

    for a in row.find_all('a'):
        text_parts.append(a.get_text(strip=True))

    for text in row.stripped_strings:
        text_parts.append(text)

    unique_text_parts = list(dict.fromkeys(text_parts))
    data_value = ' '.join(part for part in unique_text_parts if part)

    return data_value.strip()

# Define the URL to scrape
url = 'https://en.m.wikipedia.org/wiki/List_of_Falcon_9_and_Falcon_Heavy_launches_(2010%E2%80%932019)'

# Make a request to the website
response = requests.get(url)

# Parse the HTML content
soup = BeautifulSoup(response.content, 'html.parser')

# Find all tables on the page
tables = soup.find_all('table')

# Dictionary to store all tables' data
tables_dict = {}

# Iterate through each table
for table_idx, table in enumerate(tables, 1):
    print(f"\nProcessing Table {table_idx}...")

    # Get all the rows in the table body (tbody)
    rows = table.find('tbody').find_all('tr')

    # Extract headers for the current table
    header_row = rows[0]
    headers = []

    for th in header_row.find_all('th'):
        column_name = extract_column_from_header(th)
        if column_name:
            headers.append(column_name)

    # Print extracted headers
    print(f"Headers for Table {table_idx}:", headers)

    # Prepare to store row data for the current table
    table_data = []

    # Extract data rows for the current table
    for data_row in rows[1:]:
        data_values = {}

        # Check if the row contains a flight number (i.e., it has a <th> element)
        flight_no_tag = data_row.find('th')
        td_tags = data_row.find_all('td')

        # Process only rows that have a <th> (flight number) and sufficient <td> elements (usually around 9-10 data points)
        if flight_no_tag and len(td_tags) >= 9:
            # Extract Flight No. from the first <th> in the row
            flight_no = extract_column_from_data(flight_no_tag)
            data_values[headers[0]] = flight_no  # Assign Flight No. to the first column

            # Extract data from all <td> elements in the row
            for i, td in enumerate(td_tags):
                data_value = extract_column_from_data(td)
                data_values[headers[i+1]] = data_value  # Skip the first header as it's already assigned

            # Add the extracted data row to the list
            table_data.append(data_values)

    # Store the data for the current table in the dictionary
    tables_dict[f"Table_{table_idx}"] = {
        "headers": headers,
        "rows": table_data
    }

# Print the dictionary
for table_name, table_content in tables_dict.items():
    print(f"\n--- {table_name} ---")
    print("Headers:", table_content['headers'])
    for row_idx, row_data in enumerate(table_content['rows'], 1):
        print(f"Row {row_idx} Data:", row_data)
"""


Processing Table 1...
Headers for Table 1: ['Flight No.', 'Date and time ( UTC )', 'Version, booster', 'Launch site', 'Payload', 'Payload mass', 'Orbit', 'Customer', 'Launch outcome', 'Booster landing']

Processing Table 2...
Headers for Table 2: ['Flight No.', 'Date and time ( UTC )', 'Version, booster', 'Launch site', 'Payload', 'Payload mass', 'Orbit', 'Customer', 'Launch outcome', 'Booster landing']

Processing Table 3...
Headers for Table 3: ['Flight No.', 'Date and time ( UTC )', 'Version, booster', 'Launch site', 'Payload', 'Payload mass', 'Orbit', 'Customer', 'Launch outcome', 'Booster landing']

Processing Table 4...
Headers for Table 4: ['Flight No.', 'Date and time ( UTC )', 'Version, booster', 'Launch site', 'Payload', 'Payload mass', 'Orbit', 'Customer', 'Launch outcome', 'Booster landing']

Processing Table 5...
Headers for Table 5: ['Flight No.', 'Date and time ( UTC )', 'Version, booster', 'Launch site', 'Payload', 'Payload mass', 'Orbit', 'Customer', 'Launch outcome',

##2010-19 data


In [20]:
def extract_column_from_header(row):
    """
    This function returns a cleaned column name from an HTML table cell (th).
    It removes unwanted tags like <br>, <sup> but keeps the text from <a> tags and ensures proper spacing.
    """
    for tag in row.find_all(['sup']):
        tag.decompose()  # Remove the tag and its contents

    column_name = ' '.join(row.stripped_strings)

    if not column_name.strip().isdigit():
        return column_name.strip()

    return None

def extract_column_from_data(row):
    """
    This function returns cleaned data from an HTML table cell (td).
    It removes unwanted tags but keeps the text from <a> tags and ensures proper spacing.
    """
    # Remove unwanted tags
    for tag in row.find_all(['sup', 'br']):
        tag.decompose()  # Remove the tag and its contents

    # Collect the text from <a> tags and other text
    text_parts = []

    for a in row.find_all('a'):
        text_parts.append(a.get_text(strip=True))

    for text in row.stripped_strings:
        text_parts.append(text)

    unique_text_parts = list(dict.fromkeys(text_parts))
    data_value = ' '.join(part for part in unique_text_parts if part)

    return data_value.strip()

# Define the URL to scrape
url = 'https://en.m.wikipedia.org/wiki/List_of_Falcon_9_and_Falcon_Heavy_launches_(2010%E2%80%932019)'

# Make a request to the website
response = requests.get(url)

# Parse the HTML content
soup = BeautifulSoup(response.content, 'html.parser')

# Find all tables on the page
tables = soup.find_all('table')

# Dictionary to store all data in the desired format
SpaceX_data_2010_19 = {}

# Iterate through each table
for table_idx, table in enumerate(tables, 1):
    print(f"\nProcessing Table {table_idx}...")

    # Get all the rows in the table body (tbody)
    rows = table.find('tbody').find_all('tr')

    # Extract headers for the first table
    if table_idx == 1:
        header_row = rows[0]
        for th in header_row.find_all('th'):
            column_name = extract_column_from_header(th)
            if column_name:
                SpaceX_data_2010_19[column_name] = []  # Initialize each column with an empty list

        # Print extracted headers for the first table
        print(f"Headers: {list(SpaceX_data_2010_19.keys())}")

    # Extract data rows for all tables
    for data_row in rows[1:]:
        # Check if the row contains a flight number (i.e., it has a <th> element)
        flight_no_tag = data_row.find('th')
        td_tags = data_row.find_all('td')

        # Process only rows that have a <th> (flight number) and sufficient <td> elements (usually around 9-10 data points)
        if flight_no_tag and len(td_tags) >= 9:
            # Extract Flight No. from the first <th> in the row
            flight_no = extract_column_from_data(flight_no_tag)
            SpaceX_data_2010_19["Flight No."].append(flight_no)  # Append flight number to the list

            # Extract data from all <td> elements in the row
            for i, td in enumerate(td_tags):
                data_value = extract_column_from_data(td)
                column_header = list(SpaceX_data_2010_19.keys())[i + 1]  # Get the corresponding header
                SpaceX_data_2010_19[column_header].append(data_value)  # Append the value to the respective column

# Print the collected data for inspection
print("\n--- Collected SpaceX Data ---")
for header, values in SpaceX_data_2010_19.items():
    print(f"{header}: {values[:5]}")  # Print first 5 values of each column

# Save the collected data to a JSON file
file_path = '/content/drive/My Drive/SpaceX_data_2010_19.json'
with open(file_path, 'w', encoding='utf-8') as json_file:
    json.dump(SpaceX_data_2010_19, json_file, ensure_ascii=False, indent=2)

print(f"Data saved to {file_path}.")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).

Processing Table 1...
Headers: ['Flight No.', 'Date and time ( UTC )', 'Version, booster', 'Launch site', 'Payload', 'Payload mass', 'Orbit', 'Customer', 'Launch outcome', 'Booster landing']

Processing Table 2...

Processing Table 3...

Processing Table 4...

Processing Table 5...

Processing Table 6...

Processing Table 7...

--- Collected SpaceX Data ---
Flight No.: ['1', '2', '3', '4', '5']
Date and time ( UTC ): ['4 June 2010 18:45', '8 December 2010 15:43', '22 May 2012 07:44', '8 October 2012 00:35', '1 March 2013 15:10']
Version, booster: ['F9v1.0 B0003 F9 v1.0', 'F9v1.0 B0004 F9 v1.0', 'F9 v1.0 B0005', 'F9 v1.0 B0006', 'F9 v1.0 B0007']
Launch site: ['Cape Canaveral SLC‑40 ,', 'Cape Canaveral SLC‑40 ,', 'Cape Canaveral SLC‑40 ,', 'Cape Canaveral SLC‑40 ,', 'Cape Canaveral SLC‑40 ,']
Payload: ['Dragon Spacecraft Qualification Unit', 'SpaceX COTS Demo 

In [21]:
print(SpaceX_data_2010_19.keys())

dict_keys(['Flight No.', 'Date and time ( UTC )', 'Version, booster', 'Launch site', 'Payload', 'Payload mass', 'Orbit', 'Customer', 'Launch outcome', 'Booster landing'])


##2020-22 Data

In [22]:

# Define the URL to scrape
url = 'https://en.m.wikipedia.org/wiki/List_of_Falcon_9_and_Falcon_Heavy_launches_(2020%E2%80%932022)'

# Make a request to the website
response = requests.get(url)

# Parse the HTML content
soup = BeautifulSoup(response.content, 'html.parser')

# Find all tables on the page
tables = soup.find_all('table')

# Dictionary to store all data in the desired format
SpaceX_data_2020_22 = {}

# Iterate through each table
for table_idx, table in enumerate(tables, 1):
    print(f"\nProcessing Table {table_idx}...")

    # Get all the rows in the table body (tbody)
    rows = table.find('tbody').find_all('tr')

    # Extract headers for the first table
    if table_idx == 1:
        header_row = rows[0]
        for th in header_row.find_all('th'):
            column_name = extract_column_from_header(th)
            if column_name:
                SpaceX_data_2020_22[column_name] = []  # Initialize each column with an empty list

        # Print extracted headers for the first table
        print(f"Headers: {list(SpaceX_data_2020_22.keys())}")

    # Extract data rows for all tables
    for data_row in rows[1:]:
        # Check if the row contains a flight number (i.e., it has a <th> element)
        flight_no_tag = data_row.find('th')
        td_tags = data_row.find_all('td')

        # Process only rows that have a <th> (flight number) and sufficient <td> elements (usually around 9-10 data points)
        if flight_no_tag and len(td_tags) >= 9:
            # Extract Flight No. from the first <th> in the row
            flight_no = extract_column_from_data(flight_no_tag)
            SpaceX_data_2020_22["Flight No."].append(flight_no)  # Append flight number to the list

            # Extract data from all <td> elements in the row
            for i, td in enumerate(td_tags):
                data_value = extract_column_from_data(td)
                column_header = list(SpaceX_data_2020_22.keys())[i + 1]  # Get the corresponding header
                SpaceX_data_2020_22[column_header].append(data_value)  # Append the value to the respective column

# Print the collected data for inspection
print("\n--- Collected SpaceX Data ---")
for header, values in SpaceX_data_2020_22.items():
    print(f"{header}: {values[:5]}")  # Print first 5 values of each column

# Save the collected data to a JSON file
file_path = '/content/drive/My Drive/SpaceX_data_2020_22.json'
with open(file_path, 'w', encoding='utf-8') as json_file:
    json.dump(SpaceX_data_2020_22, json_file, ensure_ascii=False, indent=2)

print(f"Data saved to {file_path}.")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).

Processing Table 1...
Headers: ['Flight No.', 'Date and time ( UTC )', 'Version, booster', 'Launch site', 'Payload', 'Payload mass', 'Orbit', 'Customer', 'Launch outcome', 'Booster landing']

Processing Table 2...

Processing Table 3...

--- Collected SpaceX Data ---
Flight No.: ['78', '79', '80', '81', '82']
Date and time ( UTC ): ['7 January 2020 02:19:21', '19 January 2020 15:30', '29 January 2020 14:07', '17 February 2020 15:05', '7 March 2020 04:50']
Version, booster: ['F9 B5 B1049.4', 'F9 B5 B1046 .4', 'F9 B5 B1051.3', 'F9 B5 B1056.4', 'F9 B5 B1059.2']
Launch site: ['Cape Canaveral SLC‑40 ,', 'Kennedy LC‑39A ,', 'Cape Canaveral SLC‑40 ,', 'Cape Canaveral SLC‑40 ,', 'Cape Canaveral SLC‑40 ,']
Payload: ['Starlink Launch\xa02 : (60 satellites)', 'Crew Dragon in-flight abort test (Dragon C205.1)', 'Starlink Launch\xa03 : (60 satellites)', 'Starlink Launch\

In [23]:
print(SpaceX_data_2020_22.keys())

dict_keys(['Flight No.', 'Date and time ( UTC )', 'Version, booster', 'Launch site', 'Payload', 'Payload mass', 'Orbit', 'Customer', 'Launch outcome', 'Booster landing'])


##2024 Data to date

In [25]:

# Define the URL to scrape
url = 'https://en.m.wikipedia.org/wiki/List_of_Falcon_9_and_Falcon_Heavy_launches#'

# Make a request to the website
response = requests.get(url)

# Parse the HTML content
soup = BeautifulSoup(response.content, 'html.parser')

# Find all tables on the page
tables = soup.find_all('table')

# Dictionary to store all data in the desired format
SpaceX_data_2023_24 = {}

# Iterate through each table
for table_idx, table in enumerate(tables, 1):
    print(f"\nProcessing Table {table_idx}...")

    # Get all the rows in the table body (tbody)
    rows = table.find('tbody').find_all('tr')

    # Extract headers for the first table
    if table_idx == 1:
        header_row = rows[0]
        for th in header_row.find_all('th'):
            column_name = extract_column_from_header(th)
            if column_name:
                SpaceX_data_2023_24[column_name] = []  # Initialize each column with an empty list

        # Print extracted headers for the first table
        print(f"Headers: {list(SpaceX_data_2023_24.keys())}")

    # Extract data rows for all tables
    for data_row in rows[1:]:
        # Check if the row contains a flight number (i.e., it has a <th> element)
        flight_no_tag = data_row.find('th')
        td_tags = data_row.find_all('td')

        # Process only rows that have a <th> (flight number) and sufficient <td> elements (usually around 9-10 data points)
        if flight_no_tag and len(td_tags) >= 9:
            # Extract Flight No. from the first <th> in the row
            flight_no = extract_column_from_data(flight_no_tag)
            SpaceX_data_2023_24["Flight No."].append(flight_no)  # Append flight number to the list

            # Extract data from all <td> elements in the row
            for i, td in enumerate(td_tags):
                data_value = extract_column_from_data(td)
                column_header = list(SpaceX_data_2023_24.keys())[i + 1]  # Get the corresponding header
                SpaceX_data_2023_24[column_header].append(data_value)  # Append the value to the respective column

# Print the collected data for inspection
print("\n--- Collected SpaceX Data ---")
for header, values in SpaceX_data_2023_24.items():
    print(f"{header}: {values[:5]}")  # Print first 5 values of each column

# Save the collected data to a JSON file
file_path = '/content/drive/My Drive/SpaceX_data_2023_24.json'
with open(file_path, 'w', encoding='utf-8') as json_file:
    json.dump(SpaceX_data_2023_24, json_file, ensure_ascii=False, indent=2)

print(f"Data saved to {file_path}.")


Processing Table 1...
Headers: ['Flight No.', 'Date and time ( UTC )', 'Version, booster', 'Launch site', 'Payload', 'Payload mass', 'Orbit', 'Customer', 'Launch outcome', 'Booster landing']

Processing Table 2...

Processing Table 3...

Processing Table 4...

Processing Table 5...

--- Collected SpaceX Data ---
Flight No.: ['195', '196', 'FH 5', '197', '198']
Date and time ( UTC ): ['3 January 2023 14:56', '10 January 2023 04:50', '15 January 2023 22:56', '18 January 2023 12:24', '19 January 2023 15:43']
Version, booster: ['F9B5 B1060.15 F9 B5', 'F9B5 B1076.2 F9 B5', 'Falcon Heavy B1070 B5 (core)', 'F9B5 B1077.2 F9 B5', 'F9B5 B1075.1 F9 B5']
Launch site: ['Cape Canaveral SLC‑40 ,', 'Cape Canaveral SLC‑40 ,', 'Kennedy LC‑39A ,', 'Cape Canaveral SLC‑40 ,', 'Vandenberg SLC‑4E ,']
Payload: ['Transporter-6 (115 payload smallsat rideshare)', 'OneWeb 16 (40\xa0satellites)', 'USSF-67 CBAS-2 LDPE-3A ( & )', 'USA-343 (GPS-III SV06)', 'Starlink Group\xa02-4 : (51 satellites)']
Payload mass: ['U

## Merging The Data

In [3]:
file_path_2010_19 = '/content/drive/My Drive/SpaceX_data_2010_19.json'
file_path_2020_22 = '/content/drive/My Drive/SpaceX_data_2020_22.json'
file_path_2023_24 = '/content/drive/My Drive/SpaceX_data_2023_24.json'

df_2010_19 = pd.read_json(file_path_2010_19)
df_2020_22 = pd.read_json(file_path_2020_22)
df_2023_24 = pd.read_json(file_path_2023_24)

df_2010_19.head()

,Flight No.,Date and time ( UTC ),"Version, booster",Launch site,Payload,Payload mass,Orbit,Customer,Launch outcome,Booster landing
0,1,4 June 2010 18:45,F9v1.0 B0003 F9 v1.0,"Cape Canaveral SLC‑40 ,",Dragon Spacecraft Qualification Unit,Unknown,LEO,SpaceX,Success,Failure (parachute)
1,2,8 December 2010 15:43,F9v1.0 B0004 F9 v1.0,"Cape Canaveral SLC‑40 ,",SpaceX COTS Demo Flight 1 (Dragon C101),Unknown,LEO ISS ( ),NASA COTS ( ),Success,Failure (parachute)
2,3,22 May 2012 07:44,F9 v1.0 B0005,"Cape Canaveral SLC‑40 ,",SpaceX COTS Demo Flight 2 (Dragon C102),"525 kg (1,157 lb) (excl. Dragon mass)",LEO ISS ( ),NASA COTS ( ),Success,No attempt
3,4,8 October 2012 00:35,F9 v1.0 B0006,"Cape Canaveral SLC‑40 ,",SpaceX CRS-1 (Dragon C103),"4,700 kg (10,400 lb) (excl. Dragon mass)",LEO ISS ( ),NASA CRS ( ),Success,No attempt
4,5,1 March 2013 15:10,F9 v1.0 B0007,"Cape Canaveral SLC‑40 ,",SpaceX CRS-2 (Dragon C104),"4,877 kg (10,752 lb) (excl. Dragon mass)",LEO ISS ( ),NASA CRS ( ),Success,No attempt


In [4]:
df_2020_22.head()

,Flight No.,Date and time ( UTC ),"Version, booster",Launch site,Payload,Payload mass,Orbit,Customer,Launch outcome,Booster landing
0,78,7 January 2020 02:19:21,F9 B5 B1049.4,"Cape Canaveral SLC‑40 ,",Starlink Launch 2 : (60 satellites),"15,600 kg (34,400 lb)",LEO,SpaceX,Success,OCISLY Success ( )
1,79,19 January 2020 15:30,F9 B5 B1046 .4,"Kennedy LC‑39A ,",Crew Dragon in-flight abort test (Dragon C205.1),"12,050 kg (26,570 lb)",Sub-orbital,NASA CTS ( ),Successful simulated failure,No attempt
2,80,29 January 2020 14:07,F9 B5 B1051.3,"Cape Canaveral SLC‑40 ,",Starlink Launch 3 : (60 satellites),"15,600 kg (34,400 lb)",LEO,SpaceX,Success,OCISLY Success ( )
3,81,17 February 2020 15:05,F9 B5 B1056.4,"Cape Canaveral SLC‑40 ,",Starlink Launch 4 : (60 satellites),"15,600 kg (34,400 lb)",LEO,SpaceX,Success,OCISLY Failure ( )
4,82,7 March 2020 04:50,F9 B5 B1059.2,"Cape Canaveral SLC‑40 ,",SpaceX CRS-20 (Dragon C112.3),"1,977 kg (4,359 lb) (excl. Dragon mass)",LEO ISS ( ),NASA CRS ( ),Success,LZ‑1 Success ( )


In [5]:
df_2023_24.head()

,Flight No.,Date and time ( UTC ),"Version, booster",Launch site,Payload,Payload mass,Orbit,Customer,Launch outcome,Booster landing
0,195,3 January 2023 14:56,F9B5 B1060.15 F9 B5,"Cape Canaveral SLC‑40 ,",Transporter-6 (115 payload smallsat rideshare),Unknown,SSO,Various,Success,LZ‑1 Success ( )
1,196,10 January 2023 04:50,F9B5 B1076.2 F9 B5,"Cape Canaveral SLC‑40 ,",OneWeb 16 (40 satellites),"6,000 kg (13,000 lb)",Polar LEO,OneWeb,Success,LZ‑1 Success ( )
2,FH 5,15 January 2023 22:56,Falcon Heavy B1070 B5 (core),"Kennedy LC‑39A ,",USSF-67 CBAS-2 LDPE-3A ( & ),"~3,750 kg (8,270 lb)",GEO,USSF,Success,No attempt
3,197,18 January 2023 12:24,F9B5 B1077.2 F9 B5,"Cape Canaveral SLC‑40 ,",USA-343 (GPS-III SV06),"4,352 kg (9,595 lb)",MEO,USSF,Success,JRTI Success ( )
4,198,19 January 2023 15:43,F9B5 B1075.1 F9 B5,"Vandenberg SLC‑4E ,",Starlink Group 2-4 : (51 satellites),"15,000 kg (33,000 lb)",LEO,SpaceX,Success,OCISLY Success ( )


In [6]:

SpaceX_data = pd.concat([df_2010_19, df_2020_22, df_2023_24], ignore_index=True)
SpaceX_data.head()

,Flight No.,Date and time ( UTC ),"Version, booster",Launch site,Payload,Payload mass,Orbit,Customer,Launch outcome,Booster landing
0,1,4 June 2010 18:45,F9v1.0 B0003 F9 v1.0,"Cape Canaveral SLC‑40 ,",Dragon Spacecraft Qualification Unit,Unknown,LEO,SpaceX,Success,Failure (parachute)
1,2,8 December 2010 15:43,F9v1.0 B0004 F9 v1.0,"Cape Canaveral SLC‑40 ,",SpaceX COTS Demo Flight 1 (Dragon C101),Unknown,LEO ISS ( ),NASA COTS ( ),Success,Failure (parachute)
2,3,22 May 2012 07:44,F9 v1.0 B0005,"Cape Canaveral SLC‑40 ,",SpaceX COTS Demo Flight 2 (Dragon C102),"525 kg (1,157 lb) (excl. Dragon mass)",LEO ISS ( ),NASA COTS ( ),Success,No attempt
3,4,8 October 2012 00:35,F9 v1.0 B0006,"Cape Canaveral SLC‑40 ,",SpaceX CRS-1 (Dragon C103),"4,700 kg (10,400 lb) (excl. Dragon mass)",LEO ISS ( ),NASA CRS ( ),Success,No attempt
4,5,1 March 2013 15:10,F9 v1.0 B0007,"Cape Canaveral SLC‑40 ,",SpaceX CRS-2 (Dragon C104),"4,877 kg (10,752 lb) (excl. Dragon mass)",LEO ISS ( ),NASA CRS ( ),Success,No attempt


In [7]:
SpaceX_data.sample(10)

,Flight No.,Date and time ( UTC ),"Version, booster",Launch site,Payload,Payload mass,Orbit,Customer,Launch outcome,Booster landing
248,242,28 July 2023 04:01,F9B5 B1062.15 F9 B5,"Cape Canaveral SLC‑40 ,",Starlink Group 6-7 : (22 satellites),"~16,100 kg (35,500 lb)",LEO,SpaceX,Success,ASOG Success ( )
182,179,5 October 2022 23:10,F9 B5 B1071.5,"Vandenberg SLC‑4E ,",Starlink Group 4-29 : (52 satellites),"~16,100 kg (35,500 lb)",LEO,SpaceX,Success,OCISLY Success ( )
86,83,18 March 2020 12:16,F9 B5 B1048.5,"Kennedy LC‑39A ,",Starlink Launch 5 : (60 satellites),"15,600 kg (34,400 lb)",LEO,SpaceX,Success,OCISLY Failure ( )
74,72,12 June 2019 14:17,F9B5 B1051.2 F9 B5,"Vandenberg SLC‑4E ,",RADARSAT Constellation (3 satellites),"4,200 kg (9,300 lb)",SSO,Canadian Space Agency (CSA),Success,LZ‑4 Success ( )
91,88,30 June 2020 20:10:46,F9 B5 B1060.1,"Cape Canaveral SLC‑40 ,",GPS III 03 Matthew Henson - ( ),"4,311 kg (9,504 lb)",MEO,U.S. Space Force,Success,JRTI Success ( )
174,171,19 August 2022 19:21:20,F9 B5 B1062.9,"Cape Canaveral SLC‑40 ,",Starlink Group 4-27 : (53 satellites),"~16,250 kg (35,830 lb)",LEO,SpaceX,Success,ASOG Success ( )
317,308,10 March 2024 23:05,F9B5 B1077.11 F9 B5,"Cape Canaveral SLC‑40 ,",Starlink Group 6-43 : (23 satellites),"~17,100 kg (37,700 lb)",LEO,SpaceX,Success,JRTI Success ( )
90,87,13 June 2020 09:21,F9 B5 B1059.3,"Cape Canaveral SLC‑40 ,",Starlink Launch 8 : (58 satellites),"15,410 kg (33,970 lb)",LEO,SpaceX,Success,OCISLY Success ( )
238,232,12 June 2023 21:35,F9B5 B1071.9 F9 B5,"Vandenberg SLC‑4E ,",Transporter-8 (72 payload smallsat rideshare),Unknown,SSO,Various,Success,LZ‑4 Success ( )
152,149,21 April 2022 17:51,F9 B5 B1060.12,"Cape Canaveral SLC‑40 ,",Starlink Group 4-14 : (53 satellites),"~16,250 kg (35,830 lb)",LEO,SpaceX,Success,JRTI Success ( )


In [8]:
SpaceX_data.tail()

,Flight No.,Date and time ( UTC ),"Version, booster",Launch site,Payload,Payload mass,Orbit,Customer,Launch outcome,Booster landing
384,374,13 September 2024 01:45,F9B5 B1071.18 F9 B5,"Vandenberg SLC-4E ,",Starlink Group 9-6 : (21 satellites),"~16,700 kg (36,800 lb)",LEO,SpaceX,Success,OCISLY Success ( )
385,375,17 September 2024 22:50,F9B5 B1067.22 F9 B5,"Cape Canaveral SLC‑40 ,",Galileo FOC FM26 & FM32 -L13 ( ),"1,600 kg (3,500 lb)",MEO,ESA,Success,JRTI Success ( )
386,376,20 September 2024 13:50,F9B5 B1075.13 F9 B5,"Vandenberg SLC-4E ,",Starlink Group 9-17 : (20 satellites),"~16,300 kg (35,900 lb)",LEO,SpaceX,Success,OCISLY Success ( )
387,377,25 September 2024 04:01,F9B5 B1081.10 F9 B5,"Vandenberg SLC-4E ,",Starlink Group 9-8 : (20 satellites),"~16,300 kg (35,900 lb)",LEO,SpaceX,Success,OCISLY Success ( )
388,378,28 September 2024 17:17,F9B5 B1085.2 F9 B5,"Cape Canaveral SLC‑40 ,",Crew-9 Crew Dragon C212.4Freedom ( Crew Dragon...,"~13,000 kg (29,000 lb)",LEO ISS ( ),NASA CTS ( ),Success,LZ‑1 Success ( )


In [9]:
# Save the combined dataframe as a CSV file
csv_file_path = '/content/drive/My Drive/SpaceX_data.csv'
SpaceX_data.to_csv(csv_file_path, index=False)

print(f"Combined data saved to {csv_file_path}.")

Combined data saved to /content/drive/My Drive/SpaceX_data.csv.
